# Negamax

Last time we discussed the minimax search method for searching trees
in adversarial games.  The alternating maximizing and minimizing steps
can be replaced with the same maximizing step if we negate the values
returned each time **and** the game is truly a zero-sum game with the
utilty in terminal states for one player always being the negative of
the utility for the other player.

Here is an illustration of Negamax applied to Tic-Tac-Toe.

In [10]:
from IPython.display import IFrame
IFrame("http://www.cs.colostate.edu/~anderson/cs440/notebooks/negamax.pdf", width=800, height=600)

Here is a python implementation.

In [19]:
### Assumes that the argument 'game' is an object with the following methods
# game.getMoves()
# game.makeMove(move)    changes lookahead player
# game.unmakeMove(move)  changes lookahead player
# game.changePlayer()    changes next turn player
# game.getUtility()
# game.isOver()
# game.__str__()

inf = float('infinity')

def negamax(game, depthLeft):
    if debug:
        print('   '*(10-depthLeft), game, end='')
    # If at terminal state or depth limit, return utility value and move None
    if game.isOver() or depthLeft == 0:
        if debug:
            print('terminal value', game.getUtility())
        return game.getUtility(), None
    if debug:
        print
    # Find best move and its value from current state
    bestValue = -inf
    bestMove = None
    for move in game.getMoves():
        # Apply a move to current state
        game.makeMove(move)
        # print('trying',game)
        # Use depth-first search to find eventual utility value and back it up.
        #  Negate it because it will come back in context of next player
        value, _ = negamax(game, depthLeft-1)
        value = - value
        # Remove the move from current state, to prepare for trying a different move
        game.unmakeMove(move)
        if debug:
            print('   '*(10-depthLeft), game, "move", move,
                  "backed up value", value)
        if value > bestValue:
            # Value for this move is better than moves tried so far from this state.
            bestValue = value
            bestMove = move
            if debug:
                print("new best")
        else:
            if debug:
                print
    return bestValue, bestMove

And we can apply `negamax` to Tic-Tac-Toe using the following
`game` class definition.

In [20]:
class TTT(object):

    def __init__(self):
        self.board = [' ']*9
        self.player = 'X'
        if False:
            self.board = ['X', 'X', ' ', 'X', 'O', 'O', ' ', ' ', ' ']
            self.player = 'O'
        self.playerLookAHead = self.player

    def locations(self, c):
        return [i for i, mark in enumerate(self.board) if mark == c]

    def getMoves(self):
        moves = self.locations(' ')
        return moves

    def getUtility(self):
        whereX = self.locations('X')
        whereO = self.locations('O')
        wins = [[0, 1, 2], [3, 4, 5], [6, 7, 8],
                [0, 3, 6], [1, 4, 7], [2, 5, 8],
                [0, 4, 8], [2, 4, 6]]
        isXWon = any([all([wi in whereX for wi in w]) for w in wins])
        isOWon = any([all([wi in whereO for wi in w]) for w in wins])
        if isXWon:
            return 1 if self.playerLookAHead is 'X' else -1
        elif isOWon:
            return 1 if self.playerLookAHead is 'O' else -1
        elif ' ' not in self.board:
            return 0
        else:
            return None

    def isOver(self):
        return self.getUtility() is not None

    def makeMove(self, move):
        self.board[move] = self.playerLookAHead
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'

    def changePlayer(self):
        self.player = 'X' if self.player == 'O' else 'O'
        self.playerLookAHead = self.player

    def unmakeMove(self, move):
        self.board[move] = ' '
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'

    def __str__(self):
        s = '{}|{}|{}\n-----\n{}|{}|{}\n-----\n{}|{}|{}'.format(*self.board)
        return s

Now, let's try an example.

In [21]:
def playGameNegamax(game):
    print(game)
    while not game.isOver():
        value, move = negamax(game, 9)
        if move is None:
            print('move is None. Stopping')
            break
        game.makeMove(move)
        print("\nPlayer", game.player, "to", move, "for value", value)
        print(game)
        game.changePlayer()

In [22]:
debug = False
playGameNegamax(TTT())

 | | 
-----
 | | 
-----
 | | 

Player X to 0 for value 0
X| | 
-----
 | | 
-----
 | | 

Player O to 4 for value 0
X| | 
-----
 |O| 
-----
 | | 

Player X to 1 for value 0
X|X| 
-----
 |O| 
-----
 | | 

Player O to 2 for value 0
X|X|O
-----
 |O| 
-----
 | | 

Player X to 6 for value 0
X|X|O
-----
 |O| 
-----
X| | 

Player O to 3 for value 0
X|X|O
-----
O|O| 
-----
X| | 

Player X to 5 for value 0
X|X|O
-----
O|O|X
-----
X| | 

Player O to 7 for value 0
X|X|O
-----
O|O|X
-----
X|O| 

Player X to 8 for value 0
X|X|O
-----
O|O|X
-----
X|O|X


## Negamax with Alpha-Beta Pruning

For a negamax version, the meanings of *alpha* and *beta* must be swapped as players alternate, and their values must be negated.

In [23]:
IFrame("http://www.cs.colostate.edu/~anderson/cs440/notebooks/negamax2.pdf", width=800, height=600)

Modify Negamax to perform alpha-beta cutoffs by making these changes:

  * Two new arguments, *alpha* and *beta*, whose initial values are $-\infty$ and $\infty$. 
  * In the for loop for trying moves, negate and swap the values of *alpha* and *beta*, and the returned value from recursive calls must be negated.
  * Do early return if *bestScore* is greater than or equal to *beta*.
  * Update *alpha* to maximum of *bestScore* and current *alpha*.

# What if you cannot search to end of game?

Apply an **evaluation function** to non-terminal states.  It must
*estimate* the expected utility (function applied to terminal states
only) of the game from the current position.  

A good evaluation function
  * orders the terminal states in the same way as the utility function,
  * cannot take too much execution time (can't search the whole remaining tree!),
  * should be strongly correlated with actual expected utility.

An evaluation function is often a simple function of **features** of
the current game position.  Choice of good features is key.  Requires
considerable knowledge of the game and of good strategies.

A strict cutoff of search at a specific depth with the evaluation
function applied there, can lead to problems.  What if the advantage
in the game swings quickly just after the cutoff?  If a state can be
judged this way, then additional search such be performed for that
state (a non-**quiescent** state).

Current methods allow computers to search about 14 plies in chess.